In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager



from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select




from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException
import time
import traceback

In [4]:
import pandas as pd
from datetime import datetime


pos = 2
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][pos])

autentica['senha'] = autentica['senha'].astype(str)
autentica['senha'] = autentica['senha'].str.zfill(11)

senha = autentica['senha'][pos]

display(autentica)
name = str(autentica['ubs'][pos])
data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

raceList = ['BRANCA', 'PRETA', 'AMARELA', 'PARDA', 'INDÍGENA', 'SEM INFORMAÇÃO']


# nav = webdriver.Chrome(ChromeDriverManager().install())
# nav = webdriver.Edge(EdgeChromiumDriverManager().install())


,login,senha,ubs
0,joana,05216791418,borges
1,agda,09947741419,dois_riachos
2,amanda,05067676477,sobreira


In [5]:
def openEpHealph():
    
#     nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     nav = webdriver.Chrome(ChromeDriverManager().install())
    global nav
    nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/login")
    
    # inserindo usuário
    nav.find_element(By.ID, 'EMail').click()
    nav.find_element(By.ID, 'EMail').send_keys(login)

    # inserindo senha 
    nav.find_element(By.ID, 'Password').click()
    nav.find_element(By.ID, 'Password').send_keys(senha)

    nav.find_element(By.XPATH, '/html/body/form/button').click()

In [6]:
def people():
    
    openEpHealph()
        
    print(f'Extração iniciada as: {datetime.now()}')

    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/individuo")

#     children = nav.find_element(By.ID, 'maxAge')
#     nav.execute_script("arguments[0].setAttribute('value', '4.0')",children)
#     time.sleep(1)
    
#     hypertension = nav.find_element(By.XPATH, '//*[@id="img_hipertensaoFlag"]').click()
#     time.sleep(1)

#     diabetes = nav.find_element(By.XPATH, '//*[@id="img_diabetesFlag"]').click()
#     time.sleep(1)

    pregnant = nav.find_element(By.XPATH, '//*[@id="img_gestanteFlag"]').click()
    time.sleep(1)

    """
    race = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[3]/div[5]/span/span[1]/span/ul/li/input')
    race.click()
    time.sleep(1)
    race.send_keys(raceList[race_item])
    race.send_keys(Keys.ENTER)
    """

    time.sleep(1)

    unity = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[4]/div[1]/span/span[1]/span/span[2]')
    unity.click()
    time.sleep(1)
    nav.find_element(By.XPATH,'/html/body/span/span/span[1]/input').send_keys(Keys.ENTER)
    time.sleep(1)
    nav.find_element(By.ID, 'button').click()

    time.sleep(1)
    nav.find_element(By.ID, 'card-toggle').click()
    time.sleep(10)

In [7]:
def get_table():
    
    list_links = []
    people()
    
    print(f'Iniciando lista de links: {datetime.now()}')

    exit = False
    
    peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
    link = peoples.find_elements(By.TAG_NAME, 'a') 
    
    
    
    while exit is False:
        try:
            for a in link:
                
                list_links.append(a.get_attribute('href'))
                
            page = nav.find_element(By.CSS_SELECTOR, f"#individuo-list-datatable_next > a:nth-child(1)")

            page.click()
            time.sleep(10)                
            
        except (ElementClickInterceptedException, NoSuchElementException):
            if ElementClickInterceptedException:    
                print(ElementClickInterceptedException)
                
            if NoSuchElementException: 
                print(NoSuchElementException)
            print('acabaram as paginas')
            break
        
        except StaleElementReferenceException:
            peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
            link = peoples.find_elements(By.TAG_NAME, 'a') 

    
    list_links = list(dict.fromkeys(list_links))
    print(f'lista de links finalizada as: {datetime.now()}')
    
    return list_links    


In [8]:
def get_indivivual_info():
    
#   name sex b_date cpf cns has dm med_rec race m_name, proffisional, unity
    name, sex, b_date, cpf, cns, has, dm, med_rec, race, m_name, profissional, unity = [], [], [], [], [], [], [], [], [], [], [], []

    
    individual = get_table()
    
    print(f'Iniciando individuos : {datetime.now()}')
    for i in individual:
        
        time.sleep(1)
        
        nav.get(i)
        
        i_name = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/h5').text
        name.append(i_name)
        
        i_sex = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[1]').text
        i_sex = i_sex.split(',')
        sex.append(i_sex[0].upper())
        
        i_bdate = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[4]/td[2]').text
        i_bdate = i_bdate.split(";")
        b_date.append(i_bdate[0])
        
        linha_dois = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[1]').text
        linha_tres = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[1]').text
        
        linha_dois_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[2]').text
        linha_tres_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[2]').text
        
        if(linha_dois.upper() == "CPF"):
            cpf.append(linha_dois_text)
            
        elif(linha_tres.upper() == "CPF"):
            cpf.append(linha_tres_text)
            
        else:
            cpf.append("--")
        
        i_cns = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        i_cns = i_cns.split(':')
        
        if i_cns[0].upper() != 'CNS':
            cns.append('--')
        else:
            cns.append(i_cns[1])
        
        i_has = "sim"
        has.append(i_has)
        
        i_dm = "não"
        dm.append(i_dm)
        
        i_medrec = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[1]/td[2]').text
        med_rec.append(i_medrec)
        
        i_race = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[5]/td[2]').text
        race.append(i_race.upper())
        
        i_mname = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[6]/td[2]').text
        m_name.append(i_mname)
        
        try:
            i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[3]').text
        
        except NoSuchElementException:
            i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        
        i_uni = i_uni.split(':')
        unity.append(i_uni[1])
        
        try:
            linha_tres = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[1]').text
            linha_tres_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[2]').text
            if(linha_tres.upper() == 'PROFISSIONAL'):
                profissional.append(linha_tres_text)
        except:
            continue
        try:
            linha_quatro = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[4]/td[1]').text
            linha_quatro_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[4]/td[2]').text
            if(linha_quatro.upper() == 'PROFISSIONAL'):
                profissional.append(linha_quatro_text)
        except:
            continue
        
        try:
            linha_cinco = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[5]/td[1]').text
            linha_cinco_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[5]/td[2]').text
            if(linha_cinco.upper() == 'PROFISSIONAL'):
                profissional.append(linha_cinco_text)
        except:
            continue
        
        try:
            linha_seis = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[6]/td[1]').text
            linha_seis_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[6]/td[2]').text
            if(linha_seis.upper() == 'PROFISSIONAL'):
                profissional.append(linha_seis_text)
            
        except:
            continue
            
        time.sleep(1)

    print(f'Finalizando individuos : {datetime.now()}')

    dict_individual = {"name": name, "sex":sex, "b_date":b_date, "cpf":cpf, "cns":cns, "has":has, "dm":dm, "med_rec":med_rec, "race":race, "m_name":m_name, "profissional":profissional, "unity":unity, "link":individual}
    df_individual = pd.DataFrame(data=dict_individual)    
    
    print(f'Extração finalizada as: {datetime.now()}')
    
    nav.quit()
    
    return df_individual

# indi = get_indivivual_info()

In [9]:
pregnant = get_indivivual_info()

Extração iniciada as: 2024-02-06 15:13:55.755758
Iniciando lista de links: 2024-02-06 15:14:12.640980
<class 'selenium.common.exceptions.ElementClickInterceptedException'>
<class 'selenium.common.exceptions.NoSuchElementException'>
acabaram as paginas
lista de links finalizada as: 2024-02-06 15:14:13.002893
Iniciando individuos : 2024-02-06 15:14:13.002917
Finalizando individuos : 2024-02-06 15:16:43.082795
Extração finalizada as: 2024-02-06 15:16:43.085263


In [10]:
pregnant = pregnant[['cpf', 'name', 'b_date', 'sex', 'race', 'm_name', 'link']]
display(pregnant)


,cpf,name,b_date,sex,race,m_name,link
0,71351173448,ALINE LOPES GOMES,11/12/1997,FEMININO,PARDA,EDILEUZA PEREIRA LOPES,https://afogadosdaingazeira.ephealth.com.br/po...
1,07140569444,AMANDA GABRIELE DE MORAES BEZERRA,18/09/2001,FEMININO,PARDA,CARMEM ADRIANA LEITE BEZERRA,https://afogadosdaingazeira.ephealth.com.br/po...
2,70611992450,CLAUDIA MILLENA DA SILVA LOPES,11/09/1996,FEMININO,PARDA,HELENA LEVINO DA SILVA LOPES,https://afogadosdaingazeira.ephealth.com.br/po...
3,08955403470,CLECIA LUCENA BARBOSA,26/10/1993,FEMININO,PARDA,MARIA APARECIDA DE LUCENA,https://afogadosdaingazeira.ephealth.com.br/po...
4,06624363428,ELISSANDRA DA SILVA STRAMARO,11/04/1987,FEMININO,PARDA,ELEONORA DA SILVA RUFINO,https://afogadosdaingazeira.ephealth.com.br/po...
5,09961471440,ELISSANDRA RODRIGUES DOS SANTOS,13/05/1986,FEMININO,PARDA,EDITE RODRIGUES DA SILVA SANTOS,https://afogadosdaingazeira.ephealth.com.br/po...
6,06599328490,ERLANE DA SILVA LUIZ,29/10/1987,FEMININO,PARDA,IRENILDA MARIA DA SILVA LUIZ,https://afogadosdaingazeira.ephealth.com.br/po...
7,07841548567,ESTELITA COELHO FERREIRA,04/03/1998,FEMININO,BRANCA,MARIA DE JESUS COELHO,https://afogadosdaingazeira.ephealth.com.br/po...
8,05910795422,FABIANA BEZERRA DOS SANTOS RIBEIRO,07/03/1987,FEMININO,PARDA,GENEDITE MARCOLINO BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
9,09733733423,FABIANA NERIS MONTEIRO,09/03/1993,FEMININO,PARDA,MARIA LÚCIA NERIS MONTEIRO,https://afogadosdaingazeira.ephealth.com.br/po...


In [11]:
pregnant = pregnant[['cpf', 'name', 'b_date', 'sex', 'race', 'm_name', 'link']]
pregnant.to_excel(f'{data_e_hora}_{name}_pregnant.xlsx', index=False)